<a href="https://colab.research.google.com/github/yuuuwoo/Video_tracking788/blob/main/Siam2_failed_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

In [ ]:
os.chdir('/content/drive/MyDrive/Project')

In [ ]:
!git clone https://github.com/zzh142857/SiameseFC-tf.git

fatal: destination path 'SiameseFC-tf' already exists and is not an empty directory.


In [ ]:
os.chdir('/content/drive/MyDrive/Project/SiameseFC-tf')

In [ ]:
os.mkdir('./data')

FileExistsError: ignored

In [ ]:
os.chdir('./data')
os.mkdir('./training')
os.mkdir('./row_data')
os.mkdir('./testing')

In [ ]:
os.chdir('/content/drive/MyDrive/Project/SiameseFC-tf')

In [ ]:
import numpy as np
import cv2
import os
import os.path
from src.region_to_bbox import region_to_bbox_normalized
from random import shuffle


def prepare_shuffled_list(data, output, directory, num):
    """
        Input:
            data_folder: relative path of folder who contains all the vedio folders for training.
            tfrecord_name: nameof the tfrecord file
            output_directory: relative dir which will contain the tfrecord file
    """
    if not os.path.exists(directory) or os.path.isfile(directory):
        os.makedirs(directory)
        
    
        
    Dir = os.getcwd()
    data = os.path.join(Dir, data)
    #get a list of dirs in data_folder, in each of which contains a training vedio
    vedio_folder_list = sorted([dir for dir in os.listdir(data) if not os.path.isfile(os.path.join(data, dir))])[:num]
    vedio_index = 0
    output_list = []

    #loop through all the vedio folders
    for vedio_folder in vedio_folder_list:
        print("Reading images from " + vedio_folder);
        vedio_index += 1
        
        vedio_folder = os.path.join(data, vedio_folder)
        #get a list of dirs in data_folder, in each of which contains a training vedio
        file_list = [dir for dir in os.listdir(os.path.join(vedio_folder,'img')) if os.path.isfile(os.path.join(os.path.join(vedio_folder,'img'), dir))]
        img_list = sorted([file for file in file_list if file.endswith(".jpg")])
        print(len(img_list))
        
        gt_file_name = 'groundtruth_rect.txt'        
        print(vedio_folder,gt_file_name)
        if vedio_folder=='/content/drive/MyDrive/Project/SiameseFC-tf/data/Human4':
          gt_file_name='groundtruth_rect.2.txt'
        assert os.path.exists(os.path.join(vedio_folder, gt_file_name))
        
        gt_file = open(os.path.join(vedio_folder, gt_file_name), 'r')


        gts = gt_file.readlines()
        for i in range(len(gts)):
          line=gts[i].replace('\t',',')
          gts[i]=line
        print(len(gts))
        #check if num of ground truth equals the num of img files
        if len(gts)>len(img_list):
          gts=gts[:len(img_list)]
        elif len(img_list)>len(gts):
          img_list=img_list[:len(gts)]
        assert len(gts) == len(img_list)
        _examples = list(zip(img_list, gts))
        
        #prepare examplar z
        z = os.path.join(os.path.join(vedio_folder, 'img'), img_list[0])
        z_img = cv2.imread(z)
        z_gt = gts[0].strip("\n").split(",") 
        print(z_gt)  
        assert len(z_gt) == 4
        #x, y in ground truth is the coordinate of the topleft corner, we need to convert to the center
        z_pos_x, z_pos_y, z_target_w, z_target_h = region_to_bbox_normalized(z_gt, z_img.shape[1], z_img.shape[0])
               
        for _example in _examples[1: ]:
            assert len(_example) == 2
            gt = _example[1].strip("\n").split(",") 
            assert len(gt) == 4
            
            x = os.path.join(os.path.join(vedio_folder, 'img'), _example[0])
            x_img = cv2.imread(x)                                    
            x_pos_x, x_pos_y, x_target_w, x_target_h = region_to_bbox_normalized(gt, x_img.shape[1], x_img.shape[0]) 
            
            
            output_list.append(z + " " + x + " " + str(z_pos_x)+ " "+ str(z_pos_y)+ " "+  str(z_target_w)+ " "+ str(z_target_h)+ " "+  str(x_pos_x)+ " "+ str(x_pos_y)+ " "+  str(x_target_w)+ " "+  str(x_target_h))
            z = x
            z_pos_x, z_pos_y, z_target_w, z_target_h = x_pos_x, x_pos_y, x_target_w, x_target_h
    print("Finished loading all the training data. Start shuffling...")
    shuffle(output_list);
    with open(os.path.join(directory,output) + ".txt","w+") as f:
        for output_file in output_list:
            f.write(output_file + "\n") 
    print("Done. tfrecords file saved at " + directory)
        

    
if __name__ == "__main__":
    prepare_shuffled_list("data", "shuffled_data_list", "tfrecords", num = 35)

Reading images from Basketball
725
/content/drive/MyDrive/Project/SiameseFC-tf/data/Basketball groundtruth_rect.txt
725
['198', '214', '34', '81']
Reading images from Biker
142
/content/drive/MyDrive/Project/SiameseFC-tf/data/Biker groundtruth_rect.txt
142
['262', '94', '16', '26']
Reading images from Bird1
408
/content/drive/MyDrive/Project/SiameseFC-tf/data/Bird1 groundtruth_rect.txt
408
['450', '91', '31', '37']
Reading images from BlurBody
334
/content/drive/MyDrive/Project/SiameseFC-tf/data/BlurBody groundtruth_rect.txt
334
['400', '48', '87', '319']
Reading images from BlurCar2
585
/content/drive/MyDrive/Project/SiameseFC-tf/data/BlurCar2 groundtruth_rect.txt
585
['227', '207', '122', '99']
Reading images from BlurFace
493
/content/drive/MyDrive/Project/SiameseFC-tf/data/BlurFace groundtruth_rect.txt
493
['246', '226', '94', '114']
Reading images from BlurOwl
631
/content/drive/MyDrive/Project/SiameseFC-tf/data/BlurOwl groundtruth_rect.txt
631
['352', '197', '56', '100']
Reading 

In [ ]:
import tensorflow as tf
import cv2
import os
import os.path
from src.parse_arguments import parse_arguments


def transform2tfrecord(data_file, tfrecord_name, output_directory, resize_width, resize_height):
    """
        Input:
            data_folder: relative path of folder who contains all the vedio folders for training.
            tfrecord_name: nameof the tfrecord file
            output_directory: relative dir which will contain the tfrecord file

    """
    if not os.path.exists(output_directory) or os.path.isfile(output_directory):
        os.makedirs(output_directory)
        
    #init tfrecord file
    filename = os.path.join(output_directory, tfrecord_name + '.tfrecords')
    writer = tf.io.TFRecordWriter(filename)
        
    cur_dir = os.getcwd()
    data_folder = os.path.join("tfrecords", data_file)
    with open(data_folder, "r") as f:
        print("Generating tfrecord from " + data_folder)
        data_list = f.readlines()
        for data in data_list:
            z, x, z_pos_x, z_pos_y, z_target_w, z_target_h, x_pos_x, x_pos_y, x_target_w, x_target_h = data.strip("\n ").split(" ")
            z_pos_x = float(z_pos_x)
            z_pos_y = float(z_pos_y)
            z_target_w = float(z_target_w)
            z_target_h = float(z_target_h)
            x_pos_x = float(x_pos_x)
            x_pos_y = float(x_pos_y)
            x_target_w = float(x_target_w)
            x_target_h = float(x_target_h)
            z_img = cv2.imread(z)
            z_img = cv2.resize(z_img, (resize_width,resize_height))
            x_img = cv2.imread(x)
            x_img = cv2.resize(x_img, (resize_width,resize_height))
            z_raw = z_img.tostring()
            x_raw = x_img.tostring()
            

            example = tf.train.Example(features=tf.train.Features(feature={
                'z_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[z_raw])),
                'x_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[x_raw])),
                'z_pos_x': tf.train.Feature(float_list=tf.train.FloatList(value=[z_pos_x])),
                'z_pos_y': tf.train.Feature(float_list=tf.train.FloatList(value=[z_pos_y])),
                'z_target_w': tf.train.Feature(float_list=tf.train.FloatList(value=[z_target_w])),
                'z_target_h': tf.train.Feature(float_list=tf.train.FloatList(value=[z_target_h])),
                'x_pos_x': tf.train.Feature(float_list=tf.train.FloatList(value=[x_pos_x])),
                'x_pos_y': tf.train.Feature(float_list=tf.train.FloatList(value=[x_pos_y])),
                'x_target_w': tf.train.Feature(float_list=tf.train.FloatList(value=[x_target_w])),
                'x_target_h': tf.train.Feature(float_list=tf.train.FloatList(value=[x_target_h]))
                
            }))
            writer.write(example.SerializeToString())
        
        
 
        
    writer.close()
    print("Writer closed.")
    print(tfrecord_name + '.tfrecords'+" is written to "+output_directory)



    
if __name__ == "__main__":
    hp, evaluation, run, env, design = parse_arguments()
    transform2tfrecord("shuffled_data_list.txt", "training_dataset", "tfrecords", resize_width = design.resize_width, resize_height = design.resize_height)

Generating tfrecord from tfrecords/shuffled_data_list.txt


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.


Writer closed.
training_dataset.tfrecords is written to tfrecords


In [ ]:
import tensorflow
print(tensorflow.__version__)

2.8.0


In [ ]:
from __future__ import division
import sys
import os
os.chdir('/content/drive/MyDrive/Project/SiameseFC-tf')
import src.siamese as siam
from src.trainer import trainer
from src.parse_arguments import parse_arguments
import src.read_training_dataset


"""
    training procedure:
    1,input z, x, pos_x, pos_y, w, d and gt of x
    2,pad and crop z,x
    3,calculate score map
    4,calculate loss
    5,bp, update variable
"""


def train():
    # avoid printing TF debugging information
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
    # TODO: allow parameters from command line or leave everything in json files?
    hp, evaluation, run, env, design = parse_arguments()
    # Set size for use with tf.image.resize_images with align_corners=True.
    # For example,
    #   [1 4 7] =>   [1 2 3 4 5 6 7]    (length 3*(3-1)+1)
    # instead of
    # [1 4 7] => [1 1 2 3 4 5 6 7 7]  (length 3*3)
    final_score_sz = hp.response_up * (design.score_sz - 1) + 1
    
    # build the computational graph of Siamese fully-convolutional network
    siamNet = siam.Siamese(design.batch_size)
    # get tensors that will be used during training
    image, z_crops, x_crops, templates_z, scores, loss, train_step, distance_to_gt, summary= siamNet.build_tracking_graph_train(final_score_sz, design, env, hp)
 
    # read tfrecodfile holding all the training data
    data_reader = src.read_training_dataset.myReader(design.resize_width, design.resize_height, design.channel)
    batched_data = data_reader.read_tfrecord(os.path.join(env.tfrecord_path, env.tfrecord_filename), num_epochs = design.num_epochs, batch_size = design.batch_size)
    
    # run trainer
    trainer(hp, run, design, final_score_sz, batched_data, image, templates_z, scores, loss, train_step, distance_to_gt,  z_crops, x_crops, siamNet, summary)




if __name__ == '__main__':
    sys.exit(train())

AttributeError: ignored